In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from dotenv import load_dotenv
load_dotenv()                      # 1️⃣  make env-vars visible

from src.containers import Container   # 2️⃣  class body reads env once

import logging
logging.basicConfig(level=logging.INFO)

container = Container()            # 3️⃣  build the container

weaviate_helper      = container.weaviate_helper()
late_chunking_helper = container.late_chunking_helper()
ingest_helper = container.ingest_helper()


INFO:src.embeddings.late_chunking:Initialized LateChunkingHelper: device=cuda, max_chunk_chars=2048
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: jinaai/jina-embeddings-v3
INFO:sentence_transformers.SentenceTransformer:5 prompts are loaded, with the keys: ['retrieval.query', 'retrieval.passage', 'separation', 'classification', 'text-matching']


In [11]:
ingest_helper.ingest_knowledge()

INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"


config/knowledge/knowledge.yaml


INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/schema/Kubernetes_fundamentals "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://127.0.0.1:8080/v1/schema/Kubernetes_fundamentals "HTTP/1.1 200 OK"
INFO:src.ingest.ingest_helper:Collection 'kubernetes_fundamentals' deleted.
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/schema/Kubernetes_fundamentals "HTTP/1.1 404 Not Found"
INFO:httpx:HTTP Request: POST http://127.0.0.1:8080/v1/schema "HTTP/1.1 200 OK"
INFO:src.database.weviate:Collection 'kubernetes_fundamentals' created.
INFO:src.embeddings.late_chunking:chunk_file: processing path=knowledge\kubernetes\website-main\content\en\docs\concepts\architecture\cgroups.md
INFO:src.embeddings.late_chunking:generate_late_chu

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: 'knowledge\\kubernetes-lab-tutorial-master\\content'

In [ ]:
with weaviate_helper.connect() as client:
    client.collections.delete(name="test")

weaviate_helper.create_collection(collection_name="test", description="test description")

INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: DELETE http://127.0.0.1:8080/v1/schema/Test "HTTP/1.1 200 OK"


In [ ]:
docs, canonical_doc = late_chunking_helper.chunk_file(
    path="knowledge/kubernetes/website-main/content/en/docs/reference/kubectl/quick-reference.md",
)
docs = late_chunking_helper.generate_late_chunking_embeddings(docs, canonical_doc)

INFO:src.embeddings.late_chunking:chunk_file: processing path=knowledge/kubernetes/website-main/content/en/docs/reference/kubectl/quick-reference.md
INFO:src.embeddings.late_chunking:generate_late_chunking_embeddings: docs=23
Batches: 100%|██████████| 1/1 [00:00<00:00,  6.41it/s]
INFO:src.embeddings.late_chunking:generate_late_chunking_embeddings: completed embeddings for 23 docs


In [ ]:
weaviate_helper.batch_insert(docs, collection_name="test")

INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/schema/Test "HTTP/1.1 200 OK"


In [ ]:
query = "how it is created a secret with several keys?"
query_embedding = late_chunking_helper.generate_query_embedding(query)
rag_results = weaviate_helper.rag_query(collection_name="test", query_embedding=query_embedding, limit=100)
rag_results = [rag_result.properties for rag_result in rag_results]
len(rag_results)


INFO:src.embeddings.late_chunking:generate_query_embedding: query='how it is created a secret with several keys?'
Batches: 100%|██████████| 1/1 [00:00<00:00,  3.40it/s]
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/schema/Test "HTTP/1.1 200 OK"


23

In [ ]:
ranked_results = late_chunking_helper.re_rank(query=query, docs=rag_results)
ranked_results

INFO:src.embeddings.late_chunking:re_rank: 23 docs, top_k=None
Batches: 100%|██████████| 1/1 [00:00<00:00,  6.31it/s]
INFO:src.embeddings.late_chunking:re_rank: returning 23 ranked docs


[({'content': '## Creating objects  \nKubernetes manifests can be defined in YAML or JSON. The file extension `.yaml`,\n`.yml`, and `.json` can be used.  \n```bash\nkubectl apply -f ./my-manifest.yaml                 # create resource(s)\nkubectl apply -f ./my1.yaml -f ./my2.yaml           # create from multiple files\nkubectl apply -f ./dir                              # create resource(s) in all manifest files in dir\nkubectl apply -f https://example.com/manifest.yaml  # create resource(s) from url (Note: this is an example domain and does not contain a valid manifest)\nkubectl create deployment nginx --image=nginx       # start a single instance of nginx\n\n# create a Job which prints "Hello World"\nkubectl create job hello --image=busybox:1.28 -- echo "Hello World"\n\n# create a CronJob that prints "Hello World" every minute\nkubectl create cronjob hello --image=busybox:1.28   --schedule="*/1 * * * *" -- echo "Hello World"\n\nkubectl explain pods                           # get the

In [ ]:
from weaviate.classes.query import Filter

with weaviate_helper.connect() as client:
    collection = client.collections.get(name="test")
    response = collection.query.fetch_objects(
    filters=Filter.by_property("filename").like("*quick-reference.md*"),
    limit=3
)
response.objects

INFO:httpx:HTTP Request: GET http://127.0.0.1:8080/v1/meta "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: GET https://pypi.org/pypi/weaviate-client/json "HTTP/1.1 200 OK"
d:\Python\MasterIA\TFM\test_folder\.venv\lib\site-packages\weaviate\warnings.py:314: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(


[Object(uuid=_WeaviateUUIDInt('5dd3f31c-5e73-4bdb-b099-e7fcf1eb1acb'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'content': '## {{% heading "whatsnext" %}}  \n* Read the [kubectl overview](/docs/reference/kubectl/) and learn about [JsonPath](/docs/reference/kubectl/jsonpath).  \n* See [kubectl](/docs/reference/kubectl/kubectl/) options.  \n* Also read [kubectl Usage Conventions](/docs/reference/kubectl/conventions/) to understand how to use kubectl in reusable scripts.  \n* See more community [kubectl cheatsheets](https://github.com/dennyzhang/cheatsheet-kubernetes-A4).', 'filename': 'knowledge/kubernetes/website-main/content/en/docs/reference/kubectl/quick-reference.md', 'chunk': '23/23', 'subchunk': '1/1'}, references=None, vector={}, collection='Test'),
 Object(uuid=_WeaviateUUIDInt('dc2a97aa-b075-49f4-a7ca-1efaeb886585'), metadata=MetadataRetur